# Utilisation de l'api SNCF

Ce notebook a pour but d'utiliser l'api SNCF afin de répondre a la question suivante : 

Dans quelle gare il y a le plus de retard de train en France.

Pour utiliser l'api nous avons besoin d'une clef API qui sera stocker dans un secret.


__Librairies :__

python-dotenv

In [71]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
from datetime import datetime

Je charge ma clé API dans une variable

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")

In [3]:
def requete_api(requete):
    reponse = requests.get(requete, auth=(api_key,""))
    reponse.status_code
    reponse_json = reponse.json()
    return reponse_json

In [4]:
def ecrire_json(data,nom_fichier):
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [5]:
def main():
    pass

### Exemples

__Le détail des modes de transport couvert par l’API__

In [21]:
requete = "https://api.sncf.com/v1/coverage/sncf/commercial_modes"
reponse = requete_api(requete)

Mon fichier JSON est organisé en plusieur clés. Je récupère toutes les clés.

In [17]:
for key in reponse.keys():
    print(key)

pagination
feed_publishers
disruptions
context
commercial_modes
links


Les données qui m'interesse sont stocké dans la clé commercial_modes, je récupère donc les données de cette clé.

Je peux ensuite les charger dans un dataframe pandas

In [22]:
data = reponse.get("commercial_modes")
dataframe = pd.DataFrame(data)
dataframe.head()

,id,name
0,commercial_mode:DBS,DB SNCF
1,commercial_mode:EUROSTAR,Eurostar
2,commercial_mode:FR:Branding::45d07532-11f7-483...,SOLEA
3,commercial_mode:FR:Branding::5c5f14b9-bff2-4f9...,TER NA
4,commercial_mode:FR:Branding::6dfb1ec4-50b5-42a...,TER HDF


__Itinéraires entre Paris et Lyon__

In [24]:
requete = "https://api.sncf.com/v1/coverage/sncf/journeys?from=admin:fr:75056&to=admin:fr:69123&datetime=20240123T105527"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")
for key in reponse.keys():
    print(key)

feed_publishers
links
journeys
tickets
disruptions
terminus
context
notes
exceptions


### Exercice

Récupérer les informations sur le trajet entre Paris Gare de Lyon et Lyon Perrache le 17 novembre à 19h57

* Indice : utiliser la requête “journeys”

* Autre indice : le format de la date est AAAAMMJJTHHMMSS (Année, mois, jour, heure, minutes, secondes

Combien y a t-il d'arrêts entre ces deux gares ?

Combien de temps d'arrêt à chacune d'elles ? 

__Je recherche les stop_area :__

In [20]:
#Dans la région sncf je recherche les places contenant "lyon"
requete = "https://api.sncf.com/v1/coverage/sncf/places?q=lyon"
reponse = requete_api(requete)
ecrire_json(reponse,"liste place.json")

In [31]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
code_gare_lyon_perrache = "stop_area:SNCF:87722025"
date = "20240124T195700" 
url_base = "https://api.sncf.com/v1/coverage/sncf/journeys?"


In [39]:
requete = f"{url_base}from={code_gare_paris_gare_de_lyon}&to={code_gare_lyon_perrache}&datetime={date}"
reponse = requete_api(requete)
ecrire_json(reponse,"reponse.json")

1


In [41]:
#Nombre de voyage proposé
voyages = reponse.get("journeys")
nb_voyage = len(voyages)
print(f"il y a {nb_voyage} proposé.s")

il y a 2 proposé.s


Je veux garder uniquement le voyage le plus rapide

In [50]:
def voyage_rapide(journeys):
    '''Cette fonction parcours une liste de voyage et renvoie le voyage le plus rapide'''

    voyage_le_plus_rapide = None
    for voyage in voyages:
        if voyage["type"] == "fastest":
            voyage_le_plus_rapide = voyage
            break

    if voyage_le_plus_rapide is None:
        print("Pas de voyage fastest trouvé")
    else:
        print("voyage rapide trouvé")

    return voyage_le_plus_rapide

In [76]:
def decode_datetime(date_codee):
    #On enleve le T du code
    date_decodee = datetime.strptime(date_codee,"%Y%m%dT%H%M%S")
    return date_decodee

In [79]:
def liste_arrets(voyage):
    '''Cette fonction extrait les arrets d'un voyage et les présente dans un dataframe avec les heures de départ et d'arrivée'''

    nb_correspondance = voyage["nb_transfers"]
    print(f"Dans ce voyage il y a {nb_correspondance} correspondances")
    # Pour un voyage sans correspondances les sections sont organisé de la façon suivante :
    # type = crow_fly / type=public_transport / type = crow_fly
    # On garde la deuxième section
    # Les arrêts sont stocké sous la clé stop_date_times
    arrets = voyage["sections"][1]["stop_date_times"]
    nb_arrets = len(arrets)
    print(f"Dans ce voyage il y a {nb_arrets} arrets")

    #Je récupère mes données sous forme d'une liste
    donnees = []

    for arret in arrets:
        #J'ajoute a chaque element de ma liste un dictionnaire contenant des informations sur les arrêts
        donnees.append(dict(nom = arret["stop_point"]["name"],
                        arrive = decode_datetime(arret["arrival_date_time"]),
                        depart = decode_datetime(arret["departure_date_time"])))

    dataframe = pd.DataFrame(donnees)

    return dataframe


In [80]:
voyage = voyage_rapide(voyages)
arrets = liste_arrets(voyage)
arrets.head()

voyage rapide trouvé
Dans ce voyage il y a 0 correspondances
Dans ce voyage il y a 5 arrets


,nom,arrive,depart
0,Paris - Gare de Lyon - Hall 1 & 2,2024-01-24 21:00:00,2024-01-24 21:00:00
1,Le Creusot - Montceau-les-Mines - Montchanin TGV,2024-01-24 22:17:00,2024-01-24 22:20:00
2,Mâcon Loché TGV,2024-01-24 22:38:00,2024-01-24 22:41:00
3,Lyon Part Dieu,2024-01-24 23:10:00,2024-01-24 23:16:00
4,Lyon Perrache,2024-01-24 23:24:00,2024-01-24 23:24:00


Vous êtes un peu pressé et vous avez peur de vous tromper en arrivant à la gare car d’autres TGV partent à peu près en même temps (à partir de 19h00) de la gare de Lyon.

* Si vous demandez à l’API, combien de résultats vous donne-t-elle ?

* Quels sont les horaires de départ de ces trains ?

* Parmi ces trains, combien de trains ont pour destination finale Lyon et qui partent le 17 novembre ?

Je veux tous les trains qui partent de gare de Lyon à 19h

In [97]:
code_gare_paris_gare_de_lyon = "stop_area:SNCF:87686006"
date = "20240125T190000"
url_base = "https://api.sncf.com/v1/coverage/sncf/stop_areas"

In [119]:
requete = f"{url_base}/{code_gare_paris_gare_de_lyon}/departures?count=20&from_datetime={date}&"
reponse = requete_api(requete)
ecrire_json(reponse,"exercice_2.json")

In [120]:
departs = reponse["departures"]
ecrire_json(departs,"test.json")

In [121]:
data=[]

for depart in departs:
    
    data.append(dict(ligne = depart["route"]["name"],
                    direction = depart["route"]["direction"]["name"],
                    heure_depart =decode_datetime(depart["stop_date_time"]["departure_date_time"])))
    
dataframe = pd.DataFrame(data)
dataframe.head(20)

,ligne,direction,heure_depart
0,Paris - Gare de Lyon - Hall 1 & 2 - Saint-Étie...,Paris - Gare de Lyon - Hall 1 & 2 (Paris),2024-01-25 19:00:00
1,A,Boissy-Saint-Léger (Boissy-Saint-Léger),2024-01-25 19:00:51
2,D,Corbeil-Essonnes (Corbeil-Essonnes),2024-01-25 19:01:00
3,A,Boissy-Saint-Léger (Boissy-Saint-Léger),2024-01-25 19:01:45
4,A,Boissy-Saint-Léger (Boissy-Saint-Léger),2024-01-25 19:03:21
5,A,Boissy-Saint-Léger (Boissy-Saint-Léger),2024-01-25 19:04:15
6,A,Boissy-Saint-Léger (Boissy-Saint-Léger),2024-01-25 19:05:51
7,D,Corbeil-Essonnes (Corbeil-Essonnes),2024-01-25 19:06:00
8,A,Boissy-Saint-Léger (Boissy-Saint-Léger),2024-01-25 19:06:45
9,R,Montereau (Montereau-Fault-Yonne),2024-01-25 19:07:00
